In [2]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from nltk import FreqDist
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk import FreqDist
train = pd.read_csv('ratings_train (1).txt',sep='\t')
test = pd.read_csv('ratings_test.txt',sep='\t')

train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


# 1. 전처리

In [3]:
print('훈련용 리뷰 개수 :',len(train))
print('훈련용 리뷰 개수 :',len(test))

훈련용 리뷰 개수 : 150000
훈련용 리뷰 개수 : 50000


# 1-1. null

In [4]:
train.isnull().sum()
test.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [5]:
train = train.dropna(subset=['document'])
test = test.dropna(subset=['document'])

In [6]:
train.isnull().sum()
test.isnull().sum()

id          0
document    0
label       0
dtype: int64

# 2-1. 중복값

In [7]:
# document 열과 label 열의 중복을 제외한 값의 개수
train['document'].nunique(), train['label'].nunique()
test['document'].nunique(), test['label'].nunique()


(49157, 2)

In [8]:
train.drop_duplicates(subset=['document'], inplace=True)
test.drop_duplicates(subset=['document'], inplace=True)

C:\Users\jangmark\AppData\Local\Temp\ipykernel_2532\601824009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop_duplicates(subset=['document'], inplace=True)


In [9]:
print('훈련용 리뷰 개수 :',len(train))
print('훈련용 리뷰 개수 :',len(test))

훈련용 리뷰 개수 : 146182
훈련용 리뷰 개수 : 49157


# 1-3. 정규 표현식 

- [] : 대괄호 안에 문자들 중 한 개의 문자와 매치. 
- 한글 : [ㄱ-ㅎㅏ-ㅣ-가-힣],[가-힣]+ <- + : 앞의 문자가 최소 한 개 이상 존재합니다.
- ^ : 뒤의 문자열로 문자열이 시작됩니다.
- $ : 앞의 문자열로 문자열이 끝납니다.
- [^문] : 해당 문자를 제외한 문자를 매치합니다.

- 모듈 함수

- re.compile() : 정규표현식을 컴파일하는 함수입니다. 
- re.findall() : 문자열에서 정규 표현식과 매치되는 모든 경우의 문자열을 찾아서 리스트로 리턴합니다. 만약, 매치되는 문자열이 없다면 빈 리스트가 리턴됩니다.
- re.sub() : 문자열에서 정규 표현식과 일치하는 부분에 대해서 다른 문자열로 대체합니다.

In [10]:
# 정규표현식
pattern = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]") # 한글을 제외한 모든 글자 뺌
# ["[^a-zA-Z]"] # 영어를 제외한 모든 글자 뺌

train['document'] = train['document'].apply(lambda x: pattern.sub('', x))
test['document'] = test['document'].apply(lambda x: pattern.sub('', x))

train = train.drop(['id'],axis=1)
test = test.drop(['id'],axis=1)

C:\Users\jangmark\AppData\Local\Temp\ipykernel_2532\3658156119.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['document'] = train['document'].apply(lambda x: pattern.sub('', x))


#  1-4. 불용어

In [12]:
stop_word = pd.read_csv('stopwords_kor.txt')


In [13]:
stop_word

,stop
0,않다
1,되어다
2,되다
3,하다
4,어떻다
...,...
679,일곱
680,여덟
681,아홉
682,령


In [14]:
okt = Okt()

In [28]:
result = []
for sentence in train['document']:
    result.append(okt.nouns(sentence))

text = []
for word in result:
    if word not in stop_word['stop'].tolist():
        text.append(word)
text

KeyboardInterrupt: 

In [ ]:
NUM_WORDS = 10000
OOV = 'OOV'

In [ ]:
tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token=OOV)

# fit_on_texts
tokenizer.fit_on_texts(text)

In [ ]:
# 단어 인덱스 확인
word_index = tokenizer.word_index
word_index

{'OOV': 1,
 '영화': 2,
 '정말': 3,
 '진짜': 4,
 '이': 5,
 '점': 6,
 '것': 7,
 '연기': 8,
 '평점': 9,
 '최고': 10,
 '사람': 11,
 '드라마': 12,
 '스토리': 13,
 '왜': 14,
 '감동': 15,
 '때': 16,
 '보고': 17,
 '생각': 18,
 '말': 19,
 '시간': 20,
 '감독': 21,
 '거': 22,
 '배우': 23,
 '내용': 24,
 '그냥': 25,
 '재미': 26,
 '더': 27,
 '내': 28,
 '좀': 29,
 '뭐': 30,
 '쓰레기': 31,
 '작품': 32,
 '그': 33,
 '사랑': 34,
 '나': 35,
 '주인공': 36,
 '이건': 37,
 '하나': 38,
 '정도': 39,
 '수': 40,
 '돈': 41,
 '볼': 42,
 '액션': 43,
 '마지막': 44,
 '걸': 45,
 '다시': 46,
 '역시': 47,
 '완전': 48,
 '년': 49,
 '느낌': 50,
 '연출': 51,
 '장면': 52,
 '개': 53,
 '이야기': 54,
 '듯': 55,
 '또': 56,
 '최악': 57,
 '임': 58,
 '명작': 59,
 '남자': 60,
 '꼭': 61,
 '인생': 62,
 '지금': 63,
 '별로': 64,
 '처음': 65,
 '안': 66,
 '난': 67,
 '기억': 68,
 '알': 69,
 '못': 70,
 '이해': 71,
 '소재': 72,
 '이영화': 73,
 '한번': 74,
 '가슴': 75,
 '끝': 76,
 '매력': 77,
 '현실': 78,
 '분': 79,
 '여자': 80,
 '보기': 81,
 '결말': 82,
 '마음': 83,
 '때문': 84,
 '줄': 85,
 '가장': 86,
 '만': 87,
 '애': 88,
 '중': 89,
 '수준': 90,
 '함': 91,
 '공감': 92,
 '실망': 93,
 '번': 94,
 '

In [ ]:
# 숫자로 변환
rets = tokenizer.texts_to_sequences(text)

In [ ]:
length = [len(x) for x in rets]
Token_Length = max(length)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

rets2 = pad_sequences(rets,maxlen=Token_Length, padding='post', truncating='post')
rets2

array([[ 371,    4,  495, ...,    0,    0,    0],
       [ 413,  210,   17, ...,    0,    0,    0],
       [ 803, 2810, 1283, ...,    0,    0,    0],
       ...,
       [6779, 1236,   36, ...,    0,    0,    0],
       [6780, 6781, 6782, ...,    0,    0,    0],
       [ 102,  556,   95, ...,    0,    0,    0]])

In [ ]:
# 모델에 적용
from tensorflow.keras.models import Sequential
model = Sequential()

In [ ]:
from tensorflow.keras.layers import SimpleRNN
model.add(input_shape=(3,5))
model.add(SimpleRNN())

TypeError: add() got an unexpected keyword argument 'input_shape'

In [ ]:
word_index = len(tokenizer.word_index)
word_index

6791

In [ ]:
from tensorflow.keras.layers import  Embedding
model.add(Embedding(word_index+1,     2,       input_length=5))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 2)              13584     
                                                                 
Total params: 13,584
Trainable params: 13,584
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='mse')